In [1]:
import sys
import pandas as pd
import statsmodels.formula.api as smf
import itertools

### How to use the function to get candidate regression equations:
> #### Function get_model_list(descriptors, target, interaction='none')
> ##### Parameters:  
    descriptors: List of names of explanatory variables
    target: String of the name of the objective variable
    interaction: {'none', 'one', 'all'}
        'none': Regression equations with interaction terms are not candidates
        'one': Candidate regression equations up to and including one interaction term
        'all': Candidate up to and including regression equations with all interaction terms
> ##### Return value:
    List of candidate regression equations
>  
> #### Notation example for the regression equation  
> 'price ~ area + age' : price = (Constant) $+$ (Correlation coefficient for area) $\times$ area $+$ (Correlation coefficient for age) $\times$ age

In [2]:
# Define functions
# Function to get a list of regression equations without interaction terms
def get_model_list_interaction_none(descriptors, name): 
    models = []
    for n in range(1, len(descriptors)+1):
        for comb in itertools.combinations(descriptors, n):
            model_description = name + ' ~ ' + ' + '.join(comb)
            models.append(model_description)
            
    return models

# Function to get a list of regression equations containing one interaction term
def get_model_list_interaction_one(descriptors, name):
    models = []
    for n in range(1, len(descriptors)+1):
        for comb in itertools.combinations(descriptors, n):
            model_description = name + ' ~ ' + ' + '.join(comb)
            models.append(model_description)     
            
            interactions = []
            for interaction in itertools.combinations(comb, 2):
                interactions.append(' * '.join(interaction))
            for interaction in interactions:
                model_description = name + ' ~ ' + ' + '.join(comb) + ' + ' + interaction
                models.append(model_description)
                
    return models

# Function to get a list of regression equations including all interaction terms
def get_model_list_interaction_all(descriptors, name):
    models = []
    for n in range(1, len(descriptors)+1):
        for comb in itertools.combinations(descriptors, n):
            interactions = []
            for interaction in itertools.combinations(comb, 2):
                interactions.append(' * '.join(interaction))
            for m in range(len(interactions)+1): 
                for comb_interactions in itertools.combinations(interactions, m):
                    if len(comb_interactions)==0:
                        model_description = name + ' ~ ' + ' + '.join(comb)
                    else:
                        model_description = name + ' ~ ' + ' + '.join(comb) + ' + ' + ' + '.join(comb_interactions)
                    models.append(model_description)
                    
    return models

def get_model_list(descriptors, target, interaction='none'):
    if interaction=='none':
        models = get_model_list_interaction_none(descriptors, target)
    elif interaction=='one':
        models = get_model_list_interaction_one(descriptors, target)
    elif interaction=='all':
        models = get_model_list_interaction_all(descriptors, target)
    else:
        interactions = ['none', 'one', 'all']
        if interaction not in interactions:
            raise ValueError(f"interaction must be one of {', '.join(interactions)}")
    
    return models

In [3]:
# Upload data
# Place real_estate.csv in the same folder as this file
df = pd.read_csv('real_estate.csv')
display(df)

,area,age,walking,structure,sunlight,price
0,85,31,6,RC,Bad,13.4
1,130,22,19,RC,Good,28.2
2,85,6,4,W,Bad,29.1
3,80,9,10,W,Good,27.2
4,90,22,9,RC,Good,26.1
...,...,...,...,...,...,...
294,50,18,7,W,Good,5.0
295,85,20,12,RC,Good,29.2
296,85,11,8,RC,Bad,28.0
297,100,29,9,RC,Good,25.1


In [ ]:
# Remove outliers
# Set the data here with outliers removed:
df = 

In the following cell, you can display:

- The BIC values for the candidate regression equations.
- The results for the regression equation with the smallest BIC.

In [4]:
min_ec = sys.float_info.max
opt_model = ''
opt_results = None

# Get a list including all candidate regression equations
descriptors = ['area', 'age', 'walking', 'structure', 'sunlight']
target = 'price'
models = get_model_list(descriptors, target) # If interaction is omitted, it defaults to interaction='none'.

# Show BIC and regression equation (title line)
print('BIC\t\t formula')

# Calculate BIC for all pairs of explanatory variables and find the pair of explanatory variables with the smallest BIC
for model in models:
    # Estimate regression coefficients from candidate regression equations model and data df and obtain analysis results
    results = smf.ols(model, df).fit()
    
    # BIC for the current regression equation
    ec = results.bic

    # Show BIC and regression equation
    print('{:.0f}\t {}'.format(ec, model))
    
    # Update min_ec, opt_model, opt_results if the current BIC is less than the minimum BIC so far
    if ec < min_ec:
        min_ec = ec
        opt_model = model
        opt_results = results

# Show the regression equation and analysis results with the minimized BIC
print('\noptimum model : {}'.format(opt_model))
display(opt_results.summary())

BIC		 formula
2200	 price ~ area
2082	 price ~ age
2187	 price ~ walking
2191	 price ~ structure
2165	 price ~ sunlight
2084	 price ~ area + age
2190	 price ~ area + walking
2194	 price ~ area + structure
2168	 price ~ area + sunlight
2048	 price ~ age + walking
1912	 price ~ age + structure
2046	 price ~ age + sunlight
2186	 price ~ walking + structure
2143	 price ~ walking + sunlight
2156	 price ~ structure + sunlight
2047	 price ~ area + age + walking
1900	 price ~ area + age + structure
2045	 price ~ area + age + sunlight
2188	 price ~ area + walking + structure
2144	 price ~ area + walking + sunlight
2158	 price ~ area + structure + sunlight
1877	 price ~ age + walking + structure
1982	 price ~ age + walking + sunlight
1821	 price ~ age + structure + sunlight
2138	 price ~ walking + structure + sunlight
1858	 price ~ area + age + walking + structure
1976	 price ~ area + age + walking + sunlight
1795	 price ~ area + age + structure + sunlight
2138	 price ~ area + walking + structur

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     306.5
Date:                Mon, 11 Nov 2024   Prob (F-statistic):          4.11e-114
Time:                        00:02:09   Log-Likelihood:                -821.46
No. Observations:                 299   AIC:                             1655.
Df Residuals:                     293   BIC:                             1677.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           35.8550      1.296     27.659      0.000      33.304      38.406
structure[T.W]     -12.7353      0.559    -22.792      0.000     -13.835     -11.636
sunlight[T.Good]     7.1716      0.450     15.949      0.000       6.287       8.057
area                 0.0947      0.012      7.720      0.000       0.071       0.119
age                 -0.8291      0.025    -33.226      0.000      -0.878      -0.780
walking             -0.6095      0.050    -12.254      0.000      -0.707      -0.512
==============================================================================
Omnibus:                        7.503   Durbin-Watson:                   1.907
Prob(Omnibus):                  0.023   Jarque-Bera (JB):               10.041
Skew:                          -0.188   Prob(JB):                      0.00660
Kurtosis:                       3.815   Cond. No.                         559.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""